In [5]:
import numpy as np
import pandas as pd
import hvplot.pandas  # noqa
#For Monte Carlo
import random
#Visualization
import holoviews as hv
from tqdm import tqdm
import yfinance as yf


In [4]:
!pip install yfinance


  Using cached yfinance-0.2.4-py2.py3-none-any.whl (51 kB)
     |████████████████████████████████| 4.7 MB 1.3 MB/s eta 0:00:01
  Using cached html5lib-1.1-py2.py3-none-any.whl (112 kB)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl (499 kB)
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2021.3
    Uninstalling pytz-2021.3:
      Successfully uninstalled pytz-2021.3
  Attempting uninstall: lxml
    Found existing installation: lxml 4.8.0
    Uninstalling lxml-4.8.0:
      Successfully uninstalled lxml-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
geopandas 0.9.0 requires fiona>=1.8, which is not installed.


In [7]:
#load in ticker data for one portfolio only
stocks = ["T", "ADBE", "ABT", "CZR", "NOK", "BB", "F", "WDC"]
stock_list=yf.download(stocks, start="2010-01-01", end = "2021-01-01")


[*********************100%***********************]  8 of 8 completed


In [17]:
#Prices Dataframe
prices = stock_list["Adj Close"].dropna()





In [18]:
#returns df
returns = prices.pct_change().dropna()
returns.head()

,ABT,ADBE,BB,CZR,F,NOK,T,WDC
Date,,,,,,,,
2014-09-23 00:00:00-04:00,-0.020737,0.008780,-0.033852,-0.020690,-0.011614,-0.011614,-0.006761,-0.012017
2014-09-24 00:00:00-04:00,0.003059,0.018292,-0.004735,0.018779,0.014842,0.017626,0.003970,0.013709
2014-09-25 00:00:00-04:00,-0.014778,-0.019122,-0.067555,-0.078341,-0.012797,-0.011547,-0.009040,-0.021962
2014-09-26 00:00:00-04:00,0.000238,0.009600,0.046939,0.100000,0.008025,0.005841,0.005702,0.011436
2014-09-29 00:00:00-04:00,-0.002618,0.012580,-0.002924,-0.038636,-0.074709,-0.006968,-0.001417,-0.010690


In [19]:
#covariance dataframe
cov = returns.cov()
cov.head()

,ABT,ADBE,BB,CZR,F,NOK,T,WDC
ABT,0.000239,0.000159,0.000141,0.000149,0.000118,0.000120,0.000091,0.000162
ADBE,0.000159,0.000378,0.000218,0.000264,0.000109,0.000152,0.000083,0.000234
BB,0.000141,0.000218,0.000915,0.000383,0.000194,0.000208,0.000097,0.000296
CZR,0.000149,0.000264,0.000383,0.001561,0.000361,0.000260,0.000169,0.000471
F,0.000118,0.000109,0.000194,0.000361,0.000403,0.000157,0.000131,0.000265


In [20]:
#weights formation
np.random.seed(10) 
weights = np.random.random(len(stocks))
weights /= np.sum(weights)
weights

array([0.2000094 , 0.00538114, 0.16430988, 0.19417063, 0.12926672,
       0.05829151, 0.05135923, 0.19721149])

In [21]:
#portfolio returns
rp = (returns.mean()*252)@weights 
rp

0.18455401912039587

In [22]:
#portfolio variance
port_var = weights@(cov*252)@weights 
port_var

0.08262974869584574

In [23]:
#portfolio sharpe ratio
rf = 0.02 #risk-free rate
sharpe = (rp-rf)/np.sqrt(port_var)
sharpe

0.5724535832254747

In [27]:
#portfolio metrics function
def portfolio_metrics(weights, index='Trial'):
    
    '''
    This function generates the relative performance metrics that will be reported and will be used
    to find the optimal weights.
    
    Parameters:
    weights: initialized weights or optimal weights for performance reporting
    
    '''   
    
    rp = (returns.mean()*252)@weights 
    port_var = weights@(cov*252)@weights
    sharpe = (rp-rf)/np.sqrt(port_var)
    df = pd.DataFrame({"Expected Return": rp,
                       "Portfolio Variance":port_var,
                       'Portfolio Std': np.sqrt(port_var),
                       'Sharpe Ratio': sharpe}, index=[index])
    return df

In [25]:
np.random.seed(42)
#Empty Container
portfolios = pd.DataFrame(columns=[*stocks, "Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"])
#Loop
for i in range(10000):
    weights = np.random.random(len(stocks))
    weights /= np.sum(weights)
    portfolios.loc[i, stocks] = weights
    metrics = portfolio_metrics(weights,i)
    portfolios.loc[i, ["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]] = \
    metrics.loc[i,["Expected Return","Portfolio Variance", "Portfolio Std", "Sharpe Ratio"]]
    
portfolios

,T,ADBE,ABT,CZR,NOK,BB,F,WDC,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
0,0.096229,0.244263,0.188068,0.153811,0.040085,0.040079,0.014923,0.222543,0.225624,0.085366,0.292174,0.703771
1,0.162039,0.190871,0.005549,0.261453,0.224397,0.057239,0.049013,0.049439,0.276768,0.080132,0.283075,0.907067
2,0.102714,0.177161,0.145828,0.098321,0.206566,0.047094,0.09863,0.123686,0.165658,0.063135,0.251266,0.579694
3,0.135249,0.232846,0.059214,0.152497,0.175682,0.013775,0.180169,0.050569,0.227701,0.058588,0.242049,0.858097
4,0.015077,0.219922,0.223804,0.187362,0.0706,0.022637,0.158584,0.102014,0.228182,0.079877,0.282625,0.736601
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.061801,0.201836,0.110829,0.185273,0.057386,0.130459,0.162601,0.089815,0.221432,0.068652,0.262015,0.76878
9996,0.154924,0.214071,0.034914,0.087973,0.204321,0.05726,0.137912,0.108624,0.180344,0.054001,0.232381,0.690006
9997,0.132455,0.133074,0.11524,0.188609,0.034927,0.166358,0.097497,0.13184,0.208906,0.072582,0.26941,0.701186
9998,0.086266,0.029815,0.245079,0.079334,0.122993,0.014976,0.204899,0.216636,0.11138,0.071678,0.267727,0.341319


In [26]:
#optimal portfolio
portfolios[portfolios["Sharpe Ratio"]==portfolios["Sharpe Ratio"].max()]

,T,ADBE,ABT,CZR,NOK,BB,F,WDC,Expected Return,Portfolio Variance,Portfolio Std,Sharpe Ratio
5877,0.178029,0.304875,0.050467,0.372559,0.013595,0.016944,0.02195,0.041582,0.394508,0.107217,0.327441,1.143742
